In [ ]:
import cv2
import numpy as np

def correct_invoice_orientation(image_path, save_path=None):
    # Load image
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Binarize (adaptive thresholding for invoices)
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Invert: text should be white
    thresh = 255 - thresh

    # Morphological operations to enhance text lines
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (30, 5))
    morphed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

    # Sum pixels along axes → projection profile
    horizontal_sum = np.sum(morphed, axis=1)
    vertical_sum = np.sum(morphed, axis=0)

    # Decide orientation
    if np.max(vertical_sum) > np.max(horizontal_sum):
        print("Detected: Rotated 90° or 270°")
        rotated = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
        # Check again if upside down
        rotated_gray = cv2.cvtColor(rotated, cv2.COLOR_BGR2GRAY)
        text_density_top = np.sum(rotated_gray[:rotated_gray.shape[0]//2, :])
        text_density_bottom = np.sum(rotated_gray[rotated_gray.shape[0]//2:, :])
        if text_density_bottom > text_density_top:
            rotated = cv2.rotate(rotated, cv2.ROTATE_180)  # correct upside down
    else:
        print("Detected: Upright or upside down")
        rotated = img
        # Check if upside down
        text_density_top = np.sum(gray[:gray.shape[0]//2, :])
        text_density_bottom = np.sum(gray[gray.shape[0]//2:, :])
        if text_density_top < text_density_bottom:
            rotated = cv2.rotate(img, cv2.ROTATE_180)

    if save_path:
        cv2.imwrite(save_path, rotated)

    return rotated


# Example usage
corrected = correct_invoice_orientation("invoice.jpg", "invoice_corrected.jpg")
cv2.imshow("Corrected Invoice", corrected)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [8]:
# pip install pdf2image
# !pip install pytesseract

In [10]:
import cv2
import numpy as np
from pdf2image import convert_from_path
from PIL import Image
import pytesseract
import re


def detect_orientation_tesseract(img):
    """
    Detect page orientation using Tesseract OSD (Orientation & Script Detection).
    Returns the rotation angle (0, 90, 180, 270).
    """
    try:
        osd = pytesseract.image_to_osd(img)
        angle = int(re.search('Rotate: (\d+)', osd).group(1))
        return angle
    except Exception:
        return 0


def correct_invoice_orientation_image(img, page_num=1):
    """
    Detect and correct orientation of a single invoice image
    using projection profiles + Tesseract OSD.
    """
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    binary = 255 - binary  # text = white

    # Projection profiles
    horizontal_sum = np.sum(binary, axis=1)
    vertical_sum = np.sum(binary, axis=0)

    rotated = img
    orientation = "Upright"

    # Step 1: Projection check (sideways vs upright)
    if np.max(vertical_sum) > np.max(horizontal_sum):
        rotated = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
        orientation = "Rotated 90°"

    # Step 2: Use Tesseract OSD for final correction
    angle = detect_orientation_tesseract(rotated)
    if angle == 180:
        rotated = cv2.rotate(rotated, cv2.ROTATE_180)
        orientation = "Rotated 180°"
    elif angle == 90:
        rotated = cv2.rotate(rotated, cv2.ROTATE_90_CLOCKWISE)
        orientation = "Rotated 90°"
    elif angle == 270:
        rotated = cv2.rotate(rotated, cv2.ROTATE_90_COUNTERCLOCKWISE)
        orientation = "Rotated 270°"
    else:
        orientation = "Upright"

    print(f"📄 Page {page_num}: {orientation}")
    return rotated


def correct_pdf_orientation(pdf_path, output_path="corrected_invoice.pdf"):
    """
    Corrects orientation of each page in a PDF and saves a new PDF
    """
    # Convert PDF to list of PIL images
    pages = convert_from_path(pdf_path, dpi=300)

    corrected_images = []
    for i, page in enumerate(pages, start=1):
        # Convert PIL image -> OpenCV
        img_cv = cv2.cvtColor(np.array(page), cv2.COLOR_RGB2BGR)

        # Correct orientation
        corrected = correct_invoice_orientation_image(img_cv, page_num=i)

        # Back to PIL for saving
        corrected_pil = Image.fromarray(cv2.cvtColor(corrected, cv2.COLOR_BGR2RGB))
        corrected_images.append(corrected_pil)

    # Save all corrected pages as single PDF
    corrected_images[0].save(output_path, save_all=True, append_images=corrected_images[1:])
    print(f"\n✅ Saved corrected PDF at: {output_path}")


# Example usage
correct_pdf_orientation(
   "/Users/saikrishnakompelly/Desktop/bs/data/ocr_inputs/Indian Overseas Bank (2).pdf",
    "Indian Overseas Bank (2)_corrected.pdf"
)

correct_pdf_orientation(
    "/Users/saikrishnakompelly/Desktop/bs/data/ocr_inputs/35 Invoices copy/5.1_15.01.2024 Sterling RISQ.pdf",
    "5.1_15.01.2024 Sterling RISQ_corrected.pdf"
)


📄 Page 1: Rotated 180°
📄 Page 2: Rotated 180°

✅ Saved corrected PDF at: Indian Overseas Bank (2)_corrected.pdf
📄 Page 1: Upright
📄 Page 2: Upright

✅ Saved corrected PDF at: 5.1_15.01.2024 Sterling RISQ_corrected.pdf
